In [1]:
from elasticsearch import Elasticsearch 
es=Elasticsearch([{'host':'localhost','port':9200}])

In [2]:
import pandas as pd
import numpy as np
import time
import pyLDAvis
import pyLDAvis.gensim
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim import corpora, similarities, models, matutils
import matplotlib.pyplot as plt
import scipy.sparse as sp
import requests
import json
import psycopg2
from collections import Counter, OrderedDict
from scipy.spatial import distance
from urllib import parse


connect = psycopg2.connect(
           host = '111.111.11.1.1',
           port = 5454,
           user = 's',
           password = 'a',
           dbname = 'm')

cursor = connect.cursor()
def click(query):
    query='{} format JSON;'.format(query.rstrip())
    query=parse.quote_plus(query)
    r=requests.post('http://{}:{}/?query={}&add_http_cors_header=1'.format('111.111.11.11.11', 8888, query))
    try:
        return r.json()
    except:
        return r.text

/Users/sergeykarpovich/Anaconda/anaconda3/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/sergeykarpovich/Anaconda/anaconda3/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/sergeykarpovich/Anaconda/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/sergeykarpovich/Anaconda/anacond

OperationalError: could not translate host name "111.111.11.1.1" to address: nodename nor servname provided, or not known


In [3]:
dataStart='2020-11-01'


In [4]:
%%time
ndcgQuery=pd.DataFrame(columns=('query', 'link_url', 'link_text', 'click'))
j=0
query_url='''SELECT lowerUTF8(query) as q, link_url, link_text, count() as colvo FROM ai.clicks
WHERE toDate(  created_at ) BETWEEN  '2020-01-01' AND today()
AND link_url!=''
AND event='link_select' AND action='click'
AND block = 'serp'
AND position<5
GROUP BY lowerUTF8(query), link_url, link_text, position
ORDER BY count() DESC
limit 50000'''.format()
otvetNDCG=click(query_url)

for res in otvetNDCG.get('data'):
    j+=1
    rowt=dict(zip(['query', 'link_url', 'link_text', 'click'],\
                  [res.get('q'), res.get('link_url'), res.get('link_text'), res.get('colvo')]))
    #datetime.strptime(res.get('dd'), '%Y-%m-%d')
    row_st = pd.Series(rowt)
    row_st.name=j
    ndcgQuery = ndcgQuery.append(row_st)
        
print (len(ndcgQuery))

50000
CPU times: user 3min 28s, sys: 800 ms, total: 3min 29s
Wall time: 3min 44s


In [5]:
ndcgQuery.head()

,query,link_url,link_text,click
1,цифровой пропуск,https://nedoma.mos.ru/,Получить цифровой пропуск на передвижение по г...,850766
2,запись к врачу,https://www.mos.ru/services/zapis-k-vrachu/,"Запись на приём к врачу, отмена и перенос записи",217876
3,электронная медицинская карта,https://www.mos.ru/pgu/ru/services/link/3676/,Электронная медицинская карта,194565
4,пропуск,https://nedoma.mos.ru/,Получить цифровой пропуск на передвижение по г...,156810
5,электронный дневник,https://www.mos.ru/pgu/ru/services/procedure/0...,Электронный дневник школьника,121871


In [6]:
%%time
k=0
content=[]
stopw=['и', 'в', 'не', 'на', 'я', 'с', '—', 'ты', 'за', 'но', 'мне', '–', 'от', 'а', 'у', 'под', 'так', 'о', 'их',
      'как', 'что', 'к', 'по', 'над', 'из', 'он', 'мы', 'всё', 'о', 'без', 'твой', 'его', 'для', 'чтоб', 'во', 'это', 'да', 'если',
      'ни', 'же', 'кто', 'то', 'где', 'до', 'еще', 'бы', 'был', 'вот', 'или', 'меня', 'когда', 'там', 'мой', 'нас', 'они', '*', 'только',
      'ты', 'сам', 'о', 'со', 'нам', 'все', 'ж', 'даже', 'ли', 'тебе', 'есть', 'чем', 'хоть', 'пред', 'вся', 'и', 'вам', 'им', 'уж',]
for rok in ndcgQuery.itertuples():
    k+=1
    text=rok.query + " " + rok.link_text
    doc=[]
    for r in text.split():
        if r not in stopw:
            doc.append(r)
    content.append(doc)
#    print (rok.split())
    #if k>300:
    #    break
print (len(content))
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
        
train_data = list(create_tagged_document(content))
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=40)
# Build the Volabulary
model.build_vocab(train_data)
# Train the Doc2Vec model
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

dictionary = corpora.Dictionary(content)
corpus = [dictionary.doc2bow(text) for text in content]
lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=25, alpha='auto', eval_every=5)
#testText= 'лишь день ее вы ты '.split()
#testCorpus = [dictionary.doc2bow(testText)]
#dd=lda.get_document_topics(dictionary.doc2bow(testText))
#print (dd)

50000
CPU times: user 2min 17s, sys: 15.1 s, total: 2min 32s
Wall time: 2min 7s


In [53]:
indexList=['news', 'pgu']

In [3]:
indexList=['pgu']

In [54]:
%%time
big_data=pd.DataFrame()

for ind in indexList:
    df = pd.read_sql_query(f"SELECT id, lower(title) as title, lower(fulltext) as fulltext, tag1, tag2, tag3, tag4, url, f_published, f_deleted FROM {ind}.content WHERE f_deleted=False and date>=1598918400;",con=connect)
    df['index']=ind
    print (ind, len(df))
    big_data=big_data.append(df, ignore_index = True)
print (len(big_data))

news 3079
pgu 360
3439
CPU times: user 86 ms, sys: 295 ms, total: 381 ms
Wall time: 2.45 s


In [55]:
%%time
k=0
content=[]
stopw=['и', 'в', 'не', 'на', 'я', 'с', '—', 'ты', 'за', 'но', 'мне', '–', 'от', 'а', 'у', 'под', 'так', 'о', 'их',
      'как', 'что', 'к', 'по', 'над', 'из', 'он', 'мы', 'всё', 'о', 'без', 'твой', 'его', 'для', 'чтоб', 'во', 'это', 'да', 'если',
      'ни', 'же', 'кто', 'то', 'где', 'до', 'еще', 'бы', 'был', 'вот', 'или', 'меня', 'когда', 'там', 'мой', 'нас', 'они', '*', 'только',
      'ты', 'сам', 'о', 'со', 'нам', 'все', 'ж', 'даже', 'ли', 'тебе', 'есть', 'чем', 'хоть', 'пред', 'вся', 'и', 'вам', 'им', 'уж',]
for rok in big_data.itertuples():
    k+=1
    text=rok.title + " " + rok.fulltext + " " + rok.tag1+ " " + rok.tag2+ " " + rok.tag3+ " " + rok.tag4
    doc=[]
    for r in text.split():
        if r not in stopw:
            doc.append(r)
    content.append(doc)
#    print (rok.split())
    #if k>300:
    #    break
print (len(content))
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
        
train_data = list(create_tagged_document(content))
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=40)
# Build the Volabulary
model.build_vocab(train_data)
# Train the Doc2Vec model
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

dictionary = corpora.Dictionary(content)
corpus = [dictionary.doc2bow(text) for text in content]
lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=15, alpha='auto', eval_every=5)
#testText= 'лишь день ее вы ты '.split()
#testCorpus = [dictionary.doc2bow(testText)]
#dd=lda.get_document_topics(dictionary.doc2bow(testText))
#print (dd)

3439
CPU times: user 1min 59s, sys: 2.89 s, total: 2min 2s
Wall time: 56.6 s


In [7]:
content[1][1]

'технологии:'

In [7]:
followers_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(followers_data)

In [51]:
mapping = {
    "mappings": {
        "properties": {
            "title": {
                "type": "text" # formerly "string"
            },
            "cat": {
                "type": "text" # formerly "string"
            },
            "fulltext": {
                "type": "text"
            },
            "tm_vector": {
                "type": "dense_vector",
                "dims": 15
            },
            "tm_vector_all": {
                "type": "dense_vector",
                "dims": 15
            },
            "d2v_vector": {
                "type": "dense_vector",
                "dims": 100
            },
            "d2v_vector_all": {
                "type": "dense_vector",
                "dims": 100
            },
            "status" : {
                "type": "keyword"
            }
        }
    }
}

In [52]:
mappingQ = {
    "mappings": {
        "properties": {
            "title": {
                "type": "text" # formerly "string"
            },
            "link_url": {
                "type": "text" # formerly "string"
            },
            "fulltext": {
                "type": "text"
            },
            "tm_vector": {
                "type": "dense_vector",
                "dims": 25
            },
            "tm_vector_all": {
                "type": "dense_vector",
                "dims": 25
            },
            "d2v_vector": {
                "type": "dense_vector",
                "dims": 100
            },
            "d2v_vector_all": {
                "type": "dense_vector",
                "dims": 100
            },
            "status" : {
                "type": "keyword"
            }
        }
    }
}

In [54]:
es.indices.create(
    index="query_all",
    body=mappingQ
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'query_all'}

In [53]:
es.indices.delete(index="query_all")

{'acknowledged': True}

In [62]:
es.indices.create(
    index="pgu",
    body=mapping
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'pgu'}

In [34]:
def crc_to_vector(crc, size):
    df = pd.DataFrame(0., columns=['val'], index=np.arange(size))
    #df=0
    for i in np.arange(size):
        if dict(crc).get(i):
            df['val'][i]=dict(crc).get(i)
        else:
            df['val'][i]=0.
            
    return df.val.tolist()

In [35]:
def doc_to_vector(crc, size):
    vecDoc=[]
    dicDoc=dict(crc)
    for i in np.arange(size):
        if dicDoc.get(i):
            vecDoc.insert(i, dicDoc.get(i))
        else:
            vecDoc.insert(i, 0.)
            
    return vecDoc

In [65]:
big_data.head()

,id,title,fulltext,tag1,tag2,tag3,tag4,url,f_published,f_deleted,index
0,80127073,началось строительство рублево-архангельской л...,в москве началась подготовка к строительству с...,"метро, строительство метро, Рублево-Архангельс...",,,,https://www.mos.ru/news/item/80127073/,True,False,news
1,79189073,футбольное поле с инфраструктурой в косино-ухт...,футбольное поле с инфраструктурой в районе кос...,"Косино-Ухтомский, футбольное поле, Рафик Загру...",,,,https://www.mos.ru/news/item/79189073/,True,False,news
2,79317073,5 сентября в центре москвы временно перекроют ...,в субботу в центре москвы перекроют несколько...,"транспорт, соревнования, водители, ограничения...",,,,https://www.mos.ru/news/item/79317073/,True,False,news
3,79512073,конкурс фотографии «мой зверь 2020» вышел на м...,гпбу «мосприрода» подвело итоги конкурса фотог...,"Конкурс фотографии «Мой зверь 2020», Экология",,,,https://www.mos.ru/news/item/79512073/,True,False,news
4,79522073,уровень загрязнения воздуха в москве 08 сентяб...,вблизи автотрасс по данным на 12:00 максимальн...,"Уровень загрязнения воздуха, Экология",,,,https://www.mos.ru/news/item/79522073/,True,False,news


In [36]:
ndcgQuery.head()

,query,link_url,link_text,click
1,цифровой пропуск,https://nedoma.mos.ru/,Получить цифровой пропуск на передвижение по г...,850766
2,запись к врачу,https://www.mos.ru/services/zapis-k-vrachu/,"Запись на приём к врачу, отмена и перенос записи",217876
3,электронная медицинская карта,https://www.mos.ru/pgu/ru/services/link/3676/,Электронная медицинская карта,194565
4,пропуск,https://nedoma.mos.ru/,Получить цифровой пропуск на передвижение по г...,156810
5,электронный дневник,https://www.mos.ru/pgu/ru/services/procedure/0...,Электронный дневник школьника,121871


In [55]:
%%time
j=0
for rok in ndcgQuery.itertuples():
    textAll= rok.query+ " " + rok.link_text
    text=rok.query 
    tmVec=dict(lda.get_document_topics(dictionary.doc2bow(text.split())))
    tmVecAll=dict(lda.get_document_topics(dictionary.doc2bow(textAll.split())))
    doc2vec=model.infer_vector(text.split())
    doc2vecAll=model.infer_vector(textAll.split())
    doc = {
        "title" : rok.query,
        "fulltext": rok.link_text,
        "link_url": rok.link_url,
        "tm_vector": crc_to_vector(tmVec, 25),
        "tm_vector_all": crc_to_vector(tmVecAll, 25),
        "d2v_vector": doc2vec,
        "d2v_vector_all": doc2vecAll,
        "status": "published"
    }
    j+=1
    #print (doc)
    index_id=j+1000
    result = es.index(index="query_all", id=index_id, body=doc)
    if j%5000==0:
        print (j, result['result'])

    #if j>300:
    #    break

5000 created
10000 created
15000 created
20000 created
25000 created
30000 created
35000 created
40000 created
45000 created
50000 created
CPU times: user 6min 28s, sys: 3.68 s, total: 6min 31s
Wall time: 10min 3s


In [131]:
q="""какое метро перекроют
"""
qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
d2vec=model.infer_vector(q.split())
res= es.search(body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published" 
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'tm_vector') + 1.0", 
        "params": {
          "query_vector": doc_to_vector(qVec, 25)
        }
      }
    }
  }
})
#print ( crc_to_vector(qVec, 10) , res)
for r in res['hits']['hits']:
    if r.get('_source').get('fulltext'):
        print (r.get('_index'), r.get('_source').get('fulltext'), distance.cosine(r.get('_source').get('tm_vector'), doc_to_vector(qVec, 25)))
    else:
        print (r.get('_index'), r.get('_source').get('title'), distance.cosine(r.get('_source').get('tm_vector'), doc_to_vector(qVec, 25)))

query_all В Москве вводится ряд дополнительных ограничений 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all дн 3.1446967252435343e-10
query_all На работу и на дачу: ответы на главные вопросы о цифровых пропусках 3.1446967252435343e-10
query_all На работу и на дачу: ответы на главные вопросы о цифровых пропусках 3.1446967252435343e-10
query_all Карта «Тройка» и автомобиль — где проверить, привязаны ли они к цифровому пропуску 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all Поиск и оплата штрафов 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all Как подать заявку на оформление единовременной выплаты на детей 3.1446967252435343e-10


In [132]:
#q="""эжд
#"""
qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
d2vec=model.infer_vector(q.split())
res= es.search(body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published" 
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'tm_vector_all') + 1.0", 
        "params": {
          "query_vector": doc_to_vector(qVec, 25)
        }
      }
    }
  }
})
#print ( crc_to_vector(qVec, 10) , res)
for r in res['hits']['hits']:
    if r.get('_source').get('fulltext'):
        print (r.get('_index'), r.get('_source').get('fulltext'), distance.cosine(r.get('_source').get('tm_vector_all'), doc_to_vector(qVec, 25)))
    else:
        print (r.get('_index'), r.get('_source').get('title'), distance.cosine(r.get('_source').get('tm_vector_all'), doc_to_vector(qVec, 25)))
        

query_all  3.1446967252435343e-10
query_all дн 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all пока 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all элект 3.1446967252435343e-10
query_all  3.1446967252435343e-10
query_all  3.1446967252435343e-10


In [133]:
#q="""эжд
#"""
qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
d2vec=model.infer_vector(q.split())
res= es.search(body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published" 
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'd2v_vector') + 1.0", 
        "params": {
          "query_vector": d2vec
        }
      }
    }
  }
})
#print ( crc_to_vector(qVec, 10) , res)
for r in res['hits']['hits']:
    if r.get('_source').get('fulltext'):
        print (r.get('_index'), r.get('_source').get('fulltext'), distance.cosine(r.get('_source').get('d2v_vector'), d2vec))
    else :
        print (r.get('_index'), r.get('_source').get('title'), distance.cosine(r.get('_source').get('d2v_vector'), d2vec))

query_all Строительство станции метро «Гольяново» начнется в следующем году 0.20860379488173852
query_all Строительство станции метро «Гольяново» начнется в следующем году 0.21902639730433504
query_all Строительство станции метро «Гольяново» начнется в следующем году 0.22789461528044475
query_all метро 0.30306661171069493
query_all метро 0.3052508808206277
query_all Как проектируется Бирюлевская линия метро 0.30805508995359954
query_all Как работает московское метро 0.3246123349621779
query_all Как проектируется Бирюлевская линия метро 0.3339788111796679
query_all Получить цифровой пропуск на передвижение по городу 0.33925270082149406
query_all Проверку цифровых пропусков в московском метро планируют автоматизировать 0.35939850805746687


In [134]:
#q="""эжд
#"""
qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
d2vec=model.infer_vector(q.split())
res= es.search(body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published" 
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'd2v_vector_all') + 1.0", 
        "params": {
          "query_vector": d2vec
        }
      }
    }
  }
})
#print ( crc_to_vector(qVec, 10) , res)
for r in res['hits']['hits']:
    if r.get('_source').get('fulltext'):
        print (r.get('_index'), r.get('_source').get('fulltext'), distance.cosine(r.get('_source').get('d2v_vector_all'), d2vec))
    else:
        print (r.get('_index'), r.get('_source').get('title'), distance.cosine(r.get('_source').get('d2v_vector_all'), d2vec))

query_all Торговый комплекс с ледовой ареной и кинопарком появится у метро «Митино» — Комплекс градостроительной политики и строительства города Москвы 0.21143637565459017
query_all Как проектируется Бирюлевская линия метро 0.2323535649717443
query_all метро 0.26654745572643535
query_all Началось строительство участка метро до аэропорта Внуково 0.2733333997480484
query_all метро 0.2757981954563973
query_all Майя или МосБот: «активные граждане» выберут имя для виртуального помощника метро 0.28547502238518474
query_all Началось строительство участка метро до аэропорта Внуково 0.3131764761073855
query_all Новые ступени и тяговые цепи: на каких станциях метро отремонтировали эскалаторы 0.3319009175076746
query_all Поезда «Москва» начали курсировать на Некрасовской и Большой кольцевой линиях метро 0.3350821414898194
query_all От центра до Черемушек: как открывали Калужский радиус метро 0.34751496097842316


In [85]:
q="""из-за футбольных матчей на стадионе цска 6 мая с 16:30 и 13 мая с 11:30 до окончания мероприятий изменятся автобусные маршруты № 175 и 818.
автобус № 175 поедет от станции метро «сокол» до песчаной площади. маршрут № 818 будет курсировать на двух участках: от станции метро «динамо» до дворца спорта «мегаспорт», а также от силикатного завода до песчаной площади
"""

In [44]:
q="""из-за футбольных матчей на стадионе цска 6 мая с 16:30 и 13 мая с 11:30 до окончания мероприятий изменятся автобусные маршруты № 175 и 818.
автобус № 175 поедет от станции метро «сокол» до песчаной площади. маршрут № 818 будет курсировать на двух участках: от станции метро «динамо» до дворца спорта «мегаспорт», а также от силикатного завода до песчаной площади
"""
qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
res= es.search(index='news3',body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published" 
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'my_dense_vector') + 1.0", 
        "params": {
          "query_vector": doc_to_vector(qVec, 10)
        }
      }
    }
  }
})
#print ( crc_to_vector(qVec, 10) , res)
for r in res['hits']['hits']:
    print (r.get('_source').get('title'), distance.cosine(r.get('_source').get('my_dense_vector'), doc_to_vector(qVec, 10)))

заяц-русак замечен в серебряном бору 0.13936218821833724
на заседании совета по вопросам внутреннего контроля подвели итоги работы за 2019 год 0.14305683969819616
в районе кунцево построят новую дорогу 0.15152768694782248
трехвагонные трамваи и ремонт на красной площади: какие картины хранятся в главархиве москвы 0.17467419637403792
в природных парках столицы запели лягушки 0.18144662487778573
в московских стационарах для пациентов с covid-19 работали медики из 25 городов 0.1841611411764852
на потолках павильона «поволжье» на вднх найдена живопись 0.19508973854801193
еще шесть стационаров для лечения covid-19 вернутся к стандартному режиму работы 0.21502313282764907
в отремонтированном павильоне «семена» на вднх откроют мультимедийный парк 0.22055980305100364
в северной промзоне зеленограда увеличат протяженность водопровода 0.2315941986525294


In [27]:
q="""лыжные соревнования, зимние беговые марафоны, биатлон на коньках для взрослых и детей подготовили столичные парки, подведомственные департаменту культуры города москвы. 
 так, 14 декабря в «сокольниках» состоится костюмированный бесплатный забег  «шанхайская лыжня» . участникам предстоит пройти на беговых лыжах дистанцию протяженностью 1500 метров. присоединиться к забегу могут горожане в возрасте от 18 лет вне зависимости от физической подготовки. главное условие — в одежде должны присутствовать элементы, напоминающие о национальных костюмах, истории и культуре китая. это может быть шляпка, зонтик из рисовой бумаги, веер или маска. 
 стартовый городок расположится на 2-м лучевом просеке. там участники смогут при необходимости взять напрокат беговые лыжи, а также попробовать традиционное китайское лакомство — танхулу (фрукты в сахарном сиропе). для участия в забеге необходимо зарегистрироваться в стартовом городке. начало регистрации в 11:00, в день соревнований. 
 11 января в «сокольниках» на катке «гигант» запланированы бесплатные соревнования по  биатлону на коньках . спортсменам нужно будет пройти дистанцию протяженностью 200 метров и поразить из лазерного пистолета пять мишеней. победителем станет участник, который пробежит трассу за самое короткое время и покажет лучший результат в стрельбе по мишени. он получит памятный приз. 
 принять участие в биатлоне могут дети в возрасте от 14 лет и взрослые. при необходимости коньки можно арендовать в пункте проката возле катка «гигант». соревнования продлятся с 11:00 до 15:00, регистрация участников будет проходить на месте. 
 8 февраля измайловский парк приглашает любителей острых ощущений принять участие в  вечернем забеге по лесным тропам  shatun trail. начало — в 17:00. эта гонка названа в честь медведя-шатуна, который просыпается раньше своих сородичей и первым обследует лес. 
 участников в возрасте от 18 лет ждут три дистанции на выбор — пять, десять или 20 километров. при себе необходимо иметь налобный фонарик, так как соревнование проходит в темное время суток. участие в мероприятии платное,  регистрация на сайте парка . 
 
 в день защитника отечества, 23 февраля, в измайловском парке пройдет зимний забег « пожарная миля» . его участникам, среди которых будут сотрудники пожарной службы, необходимо будет преодолеть две дистанции на выбор. одна из них — с препятствиями (1600 метров), другая — без (пять километров). 
 принять участие в забеге могут все желающие в возрасте от 18 лет. для сотрудников пожарно-спасательной службы будет предусмотрен отдельный зачет. им придется преодолеть дистанцию в полном комплекте боевой одежды. зарегистрироваться на мероприятие и оплатить стартовый взнос для участия в забеге на 1600 метров можно  на сайте парка . забег на пять километров будет бесплатным, но необходима  предварительная регистрация . время проведения мероприятия с 09:00 до 17.00. 
 специальную новогоднюю трассу подготовят в ландшафтном парке «митино». 22 декабря там пройдет забег под названием  «9 к. митино. новогодний» . любителей спорта ждет живописный маршрут протяженностью девять километров с крутыми спусками и подъемами. каждый спортсмен получит личный номер, индивидуальный хронометр и ключ от теплой раздевалки, где можно будет оставить свои вещи. принять участие в забеге смогут горожане в возрасте от 16 до 55 лет. 
 старт запланирован на 09:00. информация о регистрации и билеты на забег появятся на сайте парка  в середине декабря . 
 1 января в этом же парке москвичи смогут принять участие в благотворительном новогоднем забеге  «подъем 1 января» . для участников подготовят четыре дистанции. детская трасса протяженностью от 500 до 1000 метров подойдет малышам в возрасте от трех лет и старше. взрослые спортсмены смогут испытать себя, приняв участие в классическом полумарафоне (21,1 километра) или в командном забеге на семь километров. 
 новичкам подойдет символическая новогодняя трасса длиной 2020 метров. спортивный праздник начнется в 12:50 и завершится в 17:00. забег будет платным, все собранные средства пойдут на покупку предметов первой необходимости нуждающимся многодетным семьям. на финише бегунов будут ждать вкусная еда, напитки и фирменные медали. информация об условиях забега появится на сайте парка  в середине декабря . 
 этой зимой будут работать 1418 городских катков. а когда в москве образуется устойчивый снежный покров, специалисты обустроят 388 лыжных трасс, 30 лыжных баз и семь горнолыжных склонов. во дворах и в парках появятся 283 ледяные горки, снежные городки, а также более 1,7 тысячи площадок для занятий спортом и зимних игр и  22 места для катания на санях и собачьих упряжках ."""

qVec=dict(lda.get_document_topics(dictionary.doc2bow(q.split())))
#print (qVec)
res= es.search(index='news2', body={
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "term" : {
              "status" : "published"
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilaritySparse(params.query_vector, 'my_sparse_vector') + 1.0",
        "params": {
          "query_vector": qVec
        }
      }
    }
  }
})
#print (qVec , res)
for r in res['hits']['hits']:
    print (r.get('_source').get('title'))
    #r.get('_source').get('my_sparse_vector')

биатлон на коньках и шанхайская лыжня: необычные спортивные соревнования в парках
билл виола, tattoo и европейский рисунок. пушкинский музей объявил планы на будущий год
на потолках павильона «поволжье» на вднх найдена живопись
сдать и не сдаться: учителя рассказали, как подготовиться к егэ
ход реставрации воспитательного дома будет проходить под общественным контролем
студенческая лига stand-up подвела итоги второго сезона
вентиляционные шахты и тоннели метро дезинфицируют каждый день
безопасная реставрация: в городе вновь начались работы по сохранению старинных зданий
террасы в горах и водопад: китай построит музей по проекту московского бюро
как получить грант на поддержку инновационных проектов


In [44]:
followers_data =  pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(followers_data)

In [38]:
import gensim
import gensim.downloader as api
# Download dataset
#dataset = api.load("text8")
#data = [d for d in dataset]

In [39]:
#data[2][1]
#dataset?

In [35]:
# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
train_data = list(create_tagged_document(content))
print(train_data[:1])

[TaggedDocument(words=['результаты', 'поквартирного', 'голосования', 'проекту', 'программы', 'реновации', 'итоги', 'голосования', 'жителей', 'проекту', 'программы', 'реновации,', 'подробное', 'описание', 'хода', 'подсчета', 'результатов', 'голосования.', 'сервис', 'доступен', 'онлайн', 'авторизации.'], tags=[0])]


In [36]:
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
# Build the Volabulary
model.build_vocab(train_data)
# Train the Doc2Vec model
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [37]:
print(model.infer_vector([ 'голосования', 'проекту', 'программы', 'реновации']))
#> array([-0.11043505,  0.21719663, -0.21167697, -0.10790558,  0.5607173 ,
#>        ...
#>        0.16428669, -0.31307793, -0.28575218, -0.0113026 ,  0.08981086],
#>       dtype=float32)

[ 0.08920857 -0.02119423  0.00968136 -0.05048368 -0.23900619 -0.13747032
 -0.06813993 -0.10552264  0.12768595 -0.1163959  -0.03782079  0.05148774
 -0.09290025  0.01077468 -0.10002856 -0.20162362 -0.05437838  0.07071388
  0.0013888  -0.06201417 -0.09645195  0.0722858   0.20132378 -0.0652304
 -0.07533055 -0.1039124   0.04238341  0.09511964 -0.10912704 -0.09266738
  0.069853   -0.10405896  0.14136986 -0.13057545 -0.12747496 -0.02122259
 -0.1394144   0.01709869  0.06987754 -0.06048527 -0.07566538 -0.02016106
  0.1260014   0.03909704 -0.10785866 -0.02910018 -0.13354021 -0.18863133
 -0.11378827  0.00050126]
